In [18]:
%%capture --no-stderr --no-display
%load_ext micropython_magic

In [19]:
# %%micropython
import os, sys

print(os.listdir())
print(sys.modules)

['abc.mpy', 'lib', 'src', 'test.py', 'typing.mpy']
{'typing': <module 'typing' from 'typing.mpy'>, 'collections': <module 'collections' from '/lib/collections/__init__.mpy'>, '__future__': <module '__future__' from '/lib/__future__.mpy'>, 'collections.abc': <module 'collections.abc' from '/lib/collections/abc.mpy'>, 'abc': <module 'abc' from 'abc.mpy'>, 'typing_extensions': <module 'typing_extensions' from '/lib/typing_extensions.mpy'>, 'rp2': <module 'rp2' from 'rp2.py'>}


In [20]:
# %%micropython

from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side

In [21]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)
try:
    y.upper()
except AttributeError as e:
    # https://docs.python.org/3/library/typing.html#typing.cast
    print(f"OK - Intentional no runtime check: {e}")

OK - Intentional no runtime check: 'int' object has no attribute 'upper'


In [22]:
# %%micropython

from typing import Dict, ParamSpec, Union, get_origin


# https://docs.python.org/3/library/typing.html#typing.get_origin

assert get_origin(str) is None, "str"

# Partial implementation of get_origin

# assert get_origin(Dict[str, int]) is dict, "Dict"
# assert get_origin(Union[int, str]) is Union, "Union"
# P = ParamSpec("P")
# assert get_origin(P.args) is P, "ParamSpec args"
# assert get_origin(P.kwargs) is P, "ParamSpec kwargs"

In [43]:
# %%micropython

# ParamSpec, 3.11 notation
# https://docs.python.org/3/library/typing.html#typing.ParamSpec

from collections.abc import Callable
from typing import TypeVar, ParamSpec

T = TypeVar('T')
P = ParamSpec('P')

def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    '''A type-safe decorator to add logging to a function.'''
    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        print(f'{f.__name__} was called')
        return f(*args, **kwargs)
    return inner

@add_logging
def add_two(x: float, y: float) -> float:
    '''Add two numbers together.'''
    return x + y

x = add_two(1, 2)
print(x)

add_two was called
3


In [23]:
# %%micropython
from typing import get_args

# partial implementation of get_args
assert get_args(int) == ()
# assert get_args(Dict[int, str]) == (int, str)
# assert get_args(Union[int, str]) == (int, str)

In [24]:
# %%micropython

from typing import no_type_check


@no_type_check
def foo(x: int) -> str:
    return x


print(foo("42"))

42


In [25]:
# %%micropython

from typing import overload


@overload
def bar(x: int) -> str: ...


@overload
def bar(x: str) -> int: ...


def bar(x):
    return x


print(bar(42))

42


In [26]:
# %%micropython

from typing import NewType

UserId = NewType("UserId", int)
some_id = UserId(524313)

print(some_id)

assert isinstance(some_id, int)

524313


In [27]:
# %%micropython

import os

try:
    os.mkdir("collections")
except OSError as e:
    pass

In [28]:
# %%micropython
import collections
print(dir(collections))
print(collections.__dict__)

['__class__', '__name__', '__dict__', '__file__', '__path__', 'abc']
{'__path__': '/lib/collections', 'abc': <module 'collections.abc' from '/lib/collections/abc.mpy'>, '__name__': 'collections', '__file__': '/lib/collections/__init__.mpy'}


In [29]:
# %%micropython
from collections.abc import Callable, Awaitable


def feeder(get_next_item: Callable[[], str]) -> None: ...  # Body


def async_query(
    on_success: Callable[[int], None], on_error: Callable[[int, Exception], None]
) -> None: ...  # Body


async def on_update(value: str) -> None: ...  # Body


callback: Callable[[str], Awaitable[None]] = on_update

# ...


def concat(x: str, y: str) -> str:
    return x + y


x: Callable[..., str]
x = str  # OK
x = concat  # Also OK


# ####

from collections.abc import Iterable
from typing import Protocol


class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...


def good_cb(*vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


batch_proc([], good_cb)  # OK

In [30]:
# %%micropython
import os

In [31]:
# %%micropython
# https://docs.python.org/3/library/typing.html#generics

from collections.abc import Mapping, Sequence


class Employee: ...


# Sequence[Employee] indicates that all elements in the sequence
# must be instances of "Employee".
# Mapping[str, str] indicates that all keys and all values in the mapping
# must be strings.
def notify_by_email(employees: Sequence[Employee], overrides: Mapping[str, str]) -> None: ...

In [32]:
# %%micropython


from collections.abc import Mapping

# Type checker will infer that all elements in ``x`` are meant to be ints
x: list[int] = []

# Type checker error: ``list`` only accepts a single type argument:
y: list[int, str] = [1, "foo"]

# Type checker will infer that all keys in ``z`` are meant to be strings,
# and that all values in ``z`` are meant to be either strings or ints
z: Mapping[str, str | int] = {}

In [33]:
# %%micropython

from typing import Generator


def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return "Done"


e = echo_round()
print(next(e))

0


In [34]:
# %%micropython

from typing import Any

a: Any = None
a = []  # OK
a = 2  # OK

s: str = ""
s = a  # OK


def foo(item: Any) -> int:
    # Passes type checking; 'item' could be any type,
    # and that type might have a 'bar' method
    item.bar()
    return 42


def hash_b(item: Any) -> int:
    try:
        # Passes type checking
        item.magic()
    except AttributeError:
        # just ignore any error for this test
        pass
    ...


# Passes type checking, since Any is compatible with all types
hash_b(42)
hash_b("foo")

In [35]:
# %%micropython

from typing import AnyStr


def concat(a: AnyStr, b: AnyStr) -> AnyStr:
    return a + b


concat("foo", "bar")  # OK, output has type 'str'
concat(b"foo", b"bar")  # OK, output has type 'bytes'
try:
    concat("foo", b"bar")  # Error, cannot mix str and bytes
except TypeError as e:
    print("OK, expected:", e)

OK, expected: unsupported types for __add__: 'str', 'bytes'


In [36]:
# %%micropython

from typing import LiteralString


def run_query(sql: LiteralString) -> None: ...


def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(f"SELECT * FROM students WHERE name = {arbitrary_string}")  # type checker error


some_str = "a" * 1000
literal_str = "drop * from tables"

caller(some_str, literal_str)

In [37]:
# %%micropython

from typing import NoReturn


def stop() -> NoReturn:
    raise RuntimeError("no way")

In [38]:
# %%micropython

from typing import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

None
None


In [48]:
# %%micropython

# ABCs for working with IO
# Generic type IO[AnyStr] and its subclasses TextIO(IO[str]) and BinaryIO(IO[bytes]) represent the types of I/O streams such as returned by open().

from typing import IO
from typing import TextIO
from typing import BinaryIO

print("TODO: Add some tests")

TODO: Add some tests


In [56]:
# %%micropython

from typing import runtime_checkable, Protocol

@runtime_checkable
class Closable(Protocol):
    def close(self): ...

try:
    assert isinstance(open('lib/typing.mpy'), Closable)
except TypeError as e:
    print(f"@runtime_checkable, not supported: {e}")


@runtime_checkable, not supported: issubclass() arg 2 must be a class or a tuple of classes


In [39]:
# %%micropython

# 3.12 type parameter syntax
# https://docs.python.org/3/reference/simple_stmts.html#the-type-statement


# type Point = tuple[float, float]

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [55]:
# %%micropython

# 3.12 type parameter syntax not supported

# from collections.abc import Sequence

# def first[T](l: Sequence[T]) -> T:  # Function is generic over the TypeVar "T"
#     return l[0]

# from collections.abc import Sequence
# from typing import TypeVar

# U = TypeVar('U')                  # Declare type variable "U"

# def second(l: Sequence[U]) -> U:  # Function is generic over the TypeVar "U"

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [54]:
# # %%micropython

# from abc import ABCMeta


# class MyABC(metaclass=ABCMeta):
#     pass

print("metaclass not supported")

metaclass not supported


In [61]:

from typing import final


class Base:
    @final
    def done(self) -> None:
        ...
class Sub(Base):
    def done(self) -> None:  # Error reported by type checker
        ...

@final
class Leaf:
    ...
class Other(Leaf):  # Error reported by type checker
    ...


other = Other()


True


MCUException: TypeError: can't create 'NoneType' instances
